# Analysis

In [1]:
import pandas as pd

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'../.keys/sctp-data-eng-ecomm-5e533c859ea1.json')

project_id = 'sctp-data-eng-ecomm'
client = bigquery.Client(credentials= credentials,project=project_id)

In [3]:
query_job = client.query("""
   SELECT *
   FROM ecomm_dev.fact_order_items
   LIMIT 1000 """)

#results = query_job.result() 

In [4]:
# Convert results to a Pandas DataFrame
df = query_job.to_dataframe()

# Display the DataFrame
print(df.head())

/opt/miniconda3/envs/elt_bigq/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                           order_id  order_item_id  \
0  d455a8cb295653b55abda06d434ab492              1   
1  7f39ba4c9052be115350065d07583cac              1   
2  9dc8d1a6f16f1b89874c29c9d8d30447              1   
3  06c07d5b2696b9631375e2877c955b3b              1   
4  5cc6d864fa9e1a90eaf5207737549ffc              3   

                         product_id                         seller_id  \
0  a2ff5a97bf95719e38ea2e3b4105bce8  0015a82c2db000af6aaaf3ae2ecb0532   
1  a2ff5a97bf95719e38ea2e3b4105bce8  0015a82c2db000af6aaaf3ae2ecb0532   
2  a2ff5a97bf95719e38ea2e3b4105bce8  0015a82c2db000af6aaaf3ae2ecb0532   
3  08574b074924071f4e201e151b152b4e  001cca7ae9ae17fb1caed9dfb1094831   
4  08574b074924071f4e201e151b152b4e  001cca7ae9ae17fb1caed9dfb1094831   

  shipping_limit_date  price  freight_value                       customer_id  \
0 2017-10-12 22:24:16  895.0          21.02  944b72539d7e1f7f7fc6e46639ef1fe3   
1 2017-10-24 23:56:20  895.0          21.02  d7fc82cbeafea77bd0a8fbbf6296e